In [46]:
import mne


fname = r"C:\Users\bci\Documents\projects\hybrid-ssvep-p300-speller\wang_experiment\record\sunsun_20230331_online\sunsun_20230331_online.fif"
raw =  mne.io.read_raw_fif(fname,  preload = True)
raw.notch_filter([50,100], trans_bandwidth = 3)
# raw.filter(5, 90, l_trans_bandwidth=2,h_trans_bandwidth=5,
#         phase='zero-double')
new_data = raw.get_data()[:,10*250:-10*250]
raw_new = mne.io.RawArray(new_data, raw.info)
montage = mne.channels.make_standard_montage('standard_1020')
raw_new.set_montage(montage)



Opening raw data file C:\Users\bci\Documents\projects\hybrid-ssvep-p300-speller\wang_experiment\record\sunsun_20230331_online\sunsun_20230331_online.fif...
    Range : 0 ... 119868 =      0.000 ...   479.472 secs
Ready.
Reading 0 ... 119868  =      0.000 ...   479.472 secs...
Setting up band-stop filter

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower transition bandwidth: 1.50 Hz
- Upper transition bandwidth: 1.50 Hz
- Filter length: 551 samples (2.204 sec)

Creating RawArray with float64 data, n_channels=9, n_times=114869
    Range : 0 ... 114868 =      0.000 ...   459.472 secs
Ready.


C:\Users\bci\AppData\Local\Temp\ipykernel_7524\1610906362.py:5: RuntimeWarning: This filename (C:\Users\bci\Documents\projects\hybrid-ssvep-p300-speller\wang_experiment\record\sunsun_20230331_online\sunsun_20230331_online.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw =  mne.io.read_raw_fif(fname,  preload = True)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    0.0s finished


<RawArray | 9 x 114869 (459.5 s), ~7.9 MB, data loaded>

In [47]:
events = mne.find_events(raw_new)
epochs = mne.Epochs(raw=raw_new,events=events, baseline=None, tmin=0.14, tmax=0.64 - 1/250, reject=None, reject_by_annotation=False)
data = epochs.get_data() 
print(data.shape)

45 events found
Event IDs: [1 2 3 4 5 6 7 8 9]
Not setting metadata
45 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 45 events and 125 original time points ...
0 bad epochs dropped
(45, 9, 125)


In [48]:
# import numpy as np
# import scipy.io

# data = data.reshape(9,5,9,1500)
# print(data.shape)
# print(data[:,:,8,:])
# data = np.transpose(data, (1, 0, 2, 3))
# data = np.transpose(data, (2, 3, 0, 1))

# data_dict = {
#     'data': data,
#     'sfreq': 250,
#     'ch_names': ['Fz', 'C3', 'Cz', 'C4', 'Pz', 'PO7', 'Oz', 'PO8', 'STIM MARKERS'],
#     'n_samples': data.shape[1]
# }
# scipy.io.savemat('sunsun.mat', data_dict)

In [49]:
X = epochs.get_data()[:, :-1, :] * 1e-6  # micro-volt default
y = epochs.events[:, -1]

Using data from preloaded Raw for 45 events and 125 original time points ...


In [50]:
print(X.shape)
print(y.shape)

(45, 8, 125)
(45,)


In [51]:
from numpy import ndarray
import numpy as np
from scipy.sparse import vstack, identity, spmatrix
from typing import Optional, cast, Union
from scipy.linalg import eigh
from scipy.sparse.linalg import eigsh
from scipy.stats import pearsonr
from sklearn.base import BaseEstimator, ClassifierMixin, TransformerMixin
from joblib import Parallel, delayed
from functools import partial



def _ged_wong(
    Z: ndarray,
    D: Optional[ndarray] = None,
    P: Optional[ndarray] = None,
    n_components=1,
    method="type1",
):
    if method != "type1" and method != "type2":
        raise ValueError("not supported method type")

    A = Z
    if D is not None:
        A = D.T @ A
    if P is not None:
        A = P.T @ A
    A = A.T @ A
    if method == "type1":
        B = Z
        if D is not None:
            B = D.T @ Z
        B = B.T @ B
        if isinstance(A, spmatrix) or isinstance(B, spmatrix):
            D, W = eigsh(A, k=n_components, M=B)
        else:
            D, W = eigh(A, B)
    elif method == "type2":
        if isinstance(A, spmatrix):
            D, W = eigsh(A, k=n_components)
        else:
            D, W = eigh(A)

    D_exist = cast(ndarray, D)
    ind = np.argsort(D_exist)[::-1]
    D_exist, W = D_exist[ind], W[:, ind]
    return D_exist[:n_components], W[:, :n_components]


def _trca_kernel(X: ndarray):
    """TRCA.
    X: (n_trials, n_channels, n_samples)
    """
    X = np.reshape(X, (-1, *X.shape[-2:]))
    M, C, N = X.shape
    n_components = C
    P = vstack([identity(N) for _ in range(M)])
    P = P @ P.T
    Z = np.hstack(X).T  # type: ignore
    _, U = _ged_wong(Z, None, P, n_components=n_components)  # U for X
    return U


def _trca_feature(
    X: ndarray,
    templates: ndarray,
    Us: ndarray,
    n_components: int = 1,
    ensemble: bool = True,
):
    rhos = []
    if not ensemble:
        for Xk, U in zip(templates, Us):
            a = U[:, :n_components].T @ X
            b = U[:, :n_components].T @ Xk
            a = np.reshape(a, (-1))
            b = np.reshape(b, (-1))
            rhos.append(pearsonr(a, b)[0])
    else:
        U = Us[:, :, :n_components]
        U = np.concatenate(U, axis=-1)
        for Xk in templates:
            a = U.T @ X
            b = U.T @ Xk
            a = np.reshape(a, (-1))
            b = np.reshape(b, (-1))
            rhos.append(pearsonr(a, b)[0])
    return rhos


class TRCA(BaseEstimator, TransformerMixin, ClassifierMixin):
    def __init__(
        self, n_components: int = 1, ensemble: bool = True, n_jobs: Optional[int] = None
    ):
        self.n_components = n_components
        self.ensemble = ensemble
        self.n_jobs = n_jobs

    def fit(self, X: ndarray, y: ndarray, Yf: Optional[ndarray] = None):
        self.classes_ = np.unique(y)
        X = np.reshape(X, (-1, *X.shape[-2:]))
        X = X - np.mean(X, axis=-1, keepdims=True)
        self.templates_ = np.stack(
            [np.mean(X[y == label], axis=0) for label in self.classes_]
        )

        self.Us_ = np.stack([_trca_kernel(X[y == label]) for label in self.classes_])
        return self

    def transform(self, X: ndarray):
        X = np.reshape(X, (-1, *X.shape[-2:]))
        X = X - np.mean(X, axis=-1, keepdims=True)
        n_components = self.n_components
        templates = self.templates_
        Us = self.Us_
        ensemble = self.ensemble
        rhos = Parallel(n_jobs=self.n_jobs)(
            delayed(
                partial(
                    _trca_feature, Us=Us, n_components=n_components, ensemble=ensemble
                )
            )(a, templates)
            for a in X
        )
        rhos = np.stack(rhos)
        return rhos

    def predict(self, X: ndarray):
        feat = self.transform(X)
        labels = self.classes_[np.argmax(feat, axis=-1)]
        return labels


def generate_cca_references(
    freqs,
    srate,
    T,
    phases: Optional[Union[ndarray, int, float]] = None,
    n_harmonics: int = 1,
):
    if isinstance(freqs, int) or isinstance(freqs, float):
        freqs = [freqs]
    freqs = np.array(freqs)[:, np.newaxis]
    if phases is None:
        phases = 0
    if isinstance(phases, int) or isinstance(phases, float):
        phases = np.array([phases])
    phases = np.array(phases)[:, np.newaxis]
    t = np.linspace(0, T, int(T * srate))

    Yf = []
    for i in range(n_harmonics):
        Yf.append(
            np.stack(
                [
                    np.sin(2 * np.pi * (i + 1) * freqs * t + np.pi * phases),
                    np.cos(2 * np.pi * (i + 1) * freqs * t + np.pi * phases),
                ],
                axis=1,
            )
        )
    Yf = np.concatenate(Yf, axis=1)
    return Yf

In [52]:
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
estimator=TRCA(n_components = 1, n_jobs=-1)
model = estimator.fit(X_train, y_train)
preds = model.predict(X_test)
acc = np.mean(preds == y_test)
print(preds)
print(y_test)
print(acc)

[5 3 2 2 9 5 9 2 4 2 2 1 5 1]
[5 8 2 9 9 5 3 7 3 7 2 6 7 1]
0.42857142857142855


In [53]:
from sklearn.model_selection import StratifiedKFold

estimator=TRCA(n_components = 1, n_jobs=-1)
skf = StratifiedKFold(n_splits=5)
skf.get_n_splits(X, y)
print(skf)
for i, (train_index, test_index) in enumerate(skf.split(X, y)):
    print(f"Fold {i}:")
    print(f"  Train: ", y[train_index])
    print(f"  Test: ", y[test_index])
    model = estimator.fit(X[train_index], y[train_index])
    preds = model.predict(X[test_index])
    acc = np.mean(preds == y[test_index])
    print(preds)
    print(y[test_index])
    print(acc)

StratifiedKFold(n_splits=5, random_state=None, shuffle=False)
Fold 0:
  Train:  [2 2 2 2 7 7 7 7 5 5 5 5 4 4 4 4 1 1 1 1 6 6 6 6 3 3 3 3 9 9 9 9 8 8 8 8]
  Test:  [2 7 5 4 1 6 3 9 8]
[2 2 5 1 7 1 3 7 3]
[2 7 5 4 1 6 3 9 8]
0.3333333333333333
Fold 1:
  Train:  [2 2 2 2 7 7 7 7 5 5 5 5 4 4 4 4 1 1 1 1 6 6 6 6 3 3 3 3 9 9 9 9 8 8 8 8]
  Test:  [2 7 5 4 1 6 3 9 8]
[2 2 5 3 6 1 3 8 9]
[2 7 5 4 1 6 3 9 8]
0.3333333333333333
Fold 2:
  Train:  [2 2 2 2 7 7 7 7 5 5 5 5 4 4 4 4 1 1 1 1 6 6 6 6 3 3 3 3 9 9 9 9 8 8 8 8]
  Test:  [2 7 5 4 1 6 3 9 8]
[2 2 2 4 6 1 1 4 8]
[2 7 5 4 1 6 3 9 8]
0.3333333333333333
Fold 3:
  Train:  [2 2 2 2 7 7 7 7 5 5 5 5 4 4 4 4 1 1 1 1 6 6 6 6 3 3 3 3 9 9 9 9 8 8 8 8]
  Test:  [2 7 5 4 1 6 3 9 8]
[2 5 5 1 1 6 3 3 1]
[2 7 5 4 1 6 3 9 8]
0.5555555555555556
Fold 4:
  Train:  [2 2 2 2 7 7 7 7 5 5 5 5 4 4 4 4 1 1 1 1 6 6 6 6 3 3 3 3 9 9 9 9 8 8 8 8]
  Test:  [2 7 5 4 1 6 3 9 8]
[7 7 5 4 6 5 1 3 1]
[2 7 5 4 1 6 3 9 8]
0.3333333333333333
